# Data

In [ ]:
%pip install synapseclient

In [ ]:
!synapse get syn51692615

Synapse username (Optional): Michal_Zajaczkowski
Auth token for user Michal_Zajaczkowski:
Welcome, Michal_Zajaczkowski!

[syn51692615]: Downloaded to /content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip


In [ ]:
%%shell
ls /content/*

/content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip

/content/sample_data:
anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


In [ ]:
%%shell
unzip /content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip -d /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData

Archive:  /content/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData.zip
   creating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/
   creating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/
  inflating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/BraTS-SSA-00121-000-seg.nii.gz  
  inflating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t1n.nii.gz  
  inflating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t2f.nii.gz  
  inflating: /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2/BraTS-SSA-00121-000/BraTS-SSA-00121-000-t2w.nii.gz  
  in

In [ ]:
!ls -R /ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData

/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData:
ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2

/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2:
BraTS-SSA-00002-000  BraTS-SSA-00056-000  BraTS-SSA-00120-000
BraTS-SSA-00007-000  BraTS-SSA-00057-000  BraTS-SSA-00121-000
BraTS-SSA-00008-000  BraTS-SSA-00068-000  BraTS-SSA-00122-000
BraTS-SSA-00010-000  BraTS-SSA-00074-000  BraTS-SSA-00127-000
BraTS-SSA-00011-000  BraTS-SSA-00076-000  BraTS-SSA-00131-000
BraTS-SSA-00012-000  BraTS-SSA-00078-000  BraTS-SSA-00133-000
BraTS-SSA-00014-000  BraTS-SSA-00080-000  BraTS-SSA-00135-000
BraTS-SSA-00015-000  BraTS-SSA-00081-000  BraTS-SSA-00141-000
BraTS-SSA-00025-000  BraTS-SSA-00092-000  BraTS-SSA-00144-000
BraTS-SSA-00026-000  BraTS-SSA-00093-000  BraTS-SSA-00146-000
BraTS-SSA-00028-000  BraTS-SSA-00095-000  BraTS-SSA-00150-000
BraTS-SSA-00037-000  BraTS-SSA-00096-000  BraTS-SSA-00152-000
BraTS-SSA-00041-000  BraTS-SSA-00097-000  BraTS-S

In [ ]:
import os
import shutil
from pathlib import Path
import json

# Ścieżki wejściowe i wyjściowe
input_dir = "/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData/ASNR-MICCAI-BraTS2023-SSA-Challenge-TrainingData_V2"
task_name = "Task02_BrainSSA"
base_output = "/content/nnUNet_raw_data_base/nnUNet_raw_data"
task_dir = os.path.join(base_output, task_name)
imagesTr_dir = os.path.join(task_dir, "imagesTr")
labelsTr_dir = os.path.join(task_dir, "labelsTr")

os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

# Mapowanie modalności
modality_map = {
    "t1c": 0,
    "t1n": 1,
    "t2f": 2,
    "t2w": 3
}

# Lista folderów pacjentów
subdirs = [d for d in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, d))]

for subdir in subdirs:
    full_path = os.path.join(input_dir, subdir)

    for file in os.listdir(full_path):
        filepath = os.path.join(full_path, file)

        if file.endswith(".nii.gz") and not "-seg.nii.gz" in file:
            # Rozpoznaj modalność
            for mod in modality_map:
                if f"-{mod}.nii.gz" in file:
                    nnunet_id = f"{subdir}_{modality_map[mod]:04}.nii.gz"
                    dest_path = os.path.join(imagesTr_dir, nnunet_id)
                    shutil.copy(filepath, dest_path)

        elif file.endswith("-seg.nii.gz"):
            label_dest = os.path.join(labelsTr_dir, f"{subdir}.nii.gz")
            shutil.copy(filepath, label_dest)

print("✅ Pliki zostały przekonwertowane do imagesTr i labelsTr.")

# Tworzenie dataset.json
dataset_json = {
    "name": "BraTS2023_SSA_Training",
    "description": "Converted BraTS2023 SSA Training data for nnUNet",
    "tensorImageSize": "4D",
    "modality": {
        "0": "T1c",
        "1": "T1n",
        "2": "T2f",
        "3": "T2w"
    },
    "labels": {
        "0": "background",
        "1": "tumor"
    },
    "numTraining": len(subdirs),
    "training": [
        {
            "image": f"./imagesTr/{subdir}_0000.nii.gz",
            "label": f"./labelsTr/{subdir}.nii.gz"
        } for subdir in subdirs
    ],
    "numTest": 0,
    "test": []
}

with open(os.path.join(task_dir, "dataset.json"), "w") as f:
    json.dump(dataset_json, f, indent=4)

print("✅ dataset.json został utworzony.")


✅ Pliki zostały przekonwertowane do imagesTr i labelsTr.
✅ dataset.json został utworzony.


✅ Pliki zostały przekonwertowane do imagesTr i labelsTr.
✅ dataset.json został utworzony.


In [ ]:
  !ls /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/

dataset.json  imagesTr	labelsTr


In [ ]:
%%capture
%%shell
git clone https://github.com/MIC-DKFZ/nnUNet.git
cd nnUNet
pip install -e .

In [ ]:
!pip install nnunet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
  Created wheel for nnunet: filename=nnunet-1.7.1-py3-none-any.whl size=531260 sha256=f40e8df5a7bcf7a137c9e44cc0bdd37106c60bc45d69e797d4e7a7195cda2fb2
  Stored in directory: /root/.cache/pip/wheels/96/24/6f/29ae4df3d7ad633127125dba95ba0e3ef10bfea39da3a0f44c
  Created wheel for medpy: filename=MedPy-0.5.2-py3-none-any.whl size=224710 sha256=e8c4ba0f1e7e6ab2544beac020941590bb19f2aedec245a5cbde459f469884ca
  Stored in directory: /root/.cache/pip/wheels/d4/33/ed/aaac5a347fb8d41679ca515b8f5c49dfdf49be15bdbb9a905d
Successfully built nnunet medpy


In [ ]:
import os

# Zmienna $HOME na Colabie to /root
os.environ["nnUNet_raw_data_base"] = "/content/nnUNet_raw_data_base"
os.environ["nnUNet_preprocessed"] = os.path.join(os.environ["HOME"], "nnUNet_preprocessed")
os.environ["RESULTS_FOLDER"] = os.path.join(os.environ["HOME"], "nnUNet_trained_models")

In [ ]:
! ls /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA

dataset.json  imagesTr	labelsTr


In [ ]:
! mkdir /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/imagesTs

In [ ]:
!nnUNet_convert_decathlon_task -i /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet



In [ ]:
%%shell
pip install -q gdown
apt-get update && apt-get install -y unrar
gdown --id 1HZmWG4j2zQg0vVwBsTrpnuLOmtKCpix2 -O /tmp/nnunet_model.rar
unrar x /tmp/nnunet_model.rar "$RESULTS_FOLDER/"
echo "✅ Model .rar rozpakowany do: $RESULTS_FOLDER"

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,969 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,245 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,944 kB]
Get:13 http://

In [ ]:
! nnUNet_print_available_pretrained_models



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

The following pretrained models are available:


Task001_BrainTumour
Brain Tumor Segmentation. 
Segmentation targets are edema, enhancing tumor and necrosis, 
Input modalities are 0: FLAIR, 1: T1, 2: T1 with contrast agent, 3: T2. 
Also see Medical Segmentation Decathlon, http://medicaldecathlon.com/

Task002_Heart
Left Atrium Segmentation. 
Segmentation target is the left atrium, 
Input modalities are 0: MRI. 
Also see Medical Segmentation Decathlon, http://medicaldecathlon.com/

Task003_Liver
Liver and Liver Tumor Segmentation. 
Segmentation targets are liver and tumors, 
Input modalities are 0: abdominal CT scan. 
Also see Medi

In [ ]:
!nnUNet_print_pretrained_model_info Task001_BrainTumour



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

Brain Tumor Segmentation. 
Segmentation targets are edema, enhancing tumor and necrosis, 
Input modalities are 0: FLAIR, 1: T1, 2: T1 with contrast agent, 3: T2. 
Also see Medical Segmentation Decathlon, http://medicaldecathlon.com/


In [ ]:
!pip install torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 894.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 110.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 758.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/1

In [ ]:
!mkdir -p /root/nnUNet_trained_models/trained_models/nnUNet/3d_fullres/Task001_BrainTumour/nnUNetTrainerV2__nnUNetPlansv2.1/
!mv /root/nnUNet_trained_models/trained_models/nnUNet/3d_fullres/Task500_BraTS2021/nnUNetTrainerV2BraTSRegions_DA4_BN_BD__nnUNetPlansv2.1/* /root/nnUNet_trained_models/trained_models/nnUNet/3d_fullres/Task001_BrainTumour/nnUNetTrainerV2__nnUNetPlansv2.1/

In [ ]:
os.environ["RESULTS_FOLDER"] = os.path.join(os.environ["HOME"], "nnUNet_trained_models", "trained_models")
#!mv /root/nnUNet_trained_models/trained_models/nnUNet/3d_fullres/Task500_BraTS2021/* $HOME/nnUNet_trained_models/nnUNet/3d_fullres/Task005_Prostate/
!nnUNet_predict -i $nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/imagesTr/ -o OUTPUT_DIRECTORY -t 1 -m 3d_fullres



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  /root/nnUNet_trained_models/trained_models/nnUNet/3d_fullres/Task001_BrainTumour/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 4 input modalities for each image
Found 60 unique case ids, here are some examples: ['BraTS-SSA-00117-000' 'BraTS-SSA-00117-000' 'BraTS-SSA-00097-000'
 'BraTS-SSA-00092-000' 'BraTS-SSA-00014-000' 'BraTS-SSA-00026-000'
 'BraTS-SSA-00012-000' 'BraTS-SSA-00150-000' 'BraTS-SSA-00127-000'
 'BraTS-SSA-00025-000']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 60
number of cases that still need to be predicted: 

In [ ]:
import nibabel as nib
import numpy as np

def load_nifti(filepath):
    return nib.load(filepath).get_fdata().astype(np.int32)

def compute_dice_per_class(gt, pred, num_classes):
    dice_scores = {}
    for c in range(num_classes):
        gt_c = (gt == c)
        pred_c = (pred == c)

        intersection = np.logical_and(gt_c, pred_c).sum()
        volume_sum = gt_c.sum() + pred_c.sum()

        if volume_sum == 0:
            dice = 1.0  # Class not present in either mask
        else:
            dice = 2.0 * intersection / volume_sum

        dice_scores[c] = dice
    return dice_scores

# File paths
gt_path = "/content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/labelsTr/BraTS-SSA-00095-000.nii.gz"
pred_path = "/content/OUTPUT_DIRECTORY/BraTS-SSA-00095-000.nii.gz"

# Load NIfTI masks
gt_mask = load_nifti(gt_path)
pred_mask = load_nifti(pred_path)

# Number of classes (e.g., 0 to N-1); change this to fit your data
num_classes = int(max(gt_mask.max(), pred_mask.max()) + 1)

# Compute Dice for each class
dice_scores = compute_dice_per_class(gt_mask, pred_mask, num_classes)

# Print results
for cls, score in dice_scores.items():
    print(f"Class {cls}: Dice Score = {score:.4f}")


Class 0: Dice Score = 0.9971
Class 1: Dice Score = 0.0997
Class 2: Dice Score = 0.0110
Class 3: Dice Score = 0.0000


Creates summary.json

In [ ]:
%%shell
nnUNet_evaluate_folder \
  -ref /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/labelsTr \
  -pred /content/OUTPUT_DIRECTORY \
  -l 1 2 3



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

OrderedDict([('all', [OrderedDict([('1', OrderedDict([('Accuracy', 0.9801961245519714), ('Dice', 0.012973600620767809), ('False Discovery Rate', 0.9932649394308236), ('False Negative Rate', 0.8240193851279721), ('False Omission Rate', 0.0006214400825997919), ('False Positive Rate', 0.01920865084246337), ('Jaccard', 0.006529153626152575), ('Negative Predictive Value', 0.9993785599174002), ('Precision', 0.0067350605691763755), ('Recall', 0.17598061487202787), ('Total Positives Reference', 6603), ('Total Positives Test', 172530), ('True Negative Rate', 0.9807913491575366)])), ('2', OrderedDict([('Accuracy', 0.9817159498207886), ('Dic

CalledProcessError: Command 'nnUNet_evaluate_folder \
  -ref /content/nnUNet_raw_data_base/nnUNet_raw_data/Task02_BrainSSA/labelsTr \
  -pred /content/OUTPUT_DIRECTORY \
  -l 1 2 3
' returned non-zero exit status 1.